In [1]:
%load_ext autoreload
%autoreload 2
import sys
import gc

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val, submissions
%matplotlib inline

In [2]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [3]:
batch_size = 1024
pred_len = 60

In [4]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_1.csv').fillna(0)

In [5]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

Use all the data to train this time.

In [6]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

Its crucial that the `testloader` is not shuffled, as we'll use the page names directly from the train csv to index them later.

In [27]:
testloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:,:]).float(),
        torch.zeros(X.shape[0], pred_len, X.shape[2])
    ),
    batch_size=batch_size, shuffle=False
)

In [8]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

Epochs chosen by amount of epochs to best val loss on validated data.

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
with clock():
    model.fit(trainloader, None, optimizer=optimizer, num_epochs=18)
save_best_path = base_dir+'rnn_v2_predictor_lr1_weights.mdl'
torch.save(model.state_dict(), save_best_path)


EPOCH 1
Running average loss: 0.435974
Elapsed time 132.3519835472107 seconds

EPOCH 2
Running average loss: 0.403352
Elapsed time 105.45952820777893 seconds

EPOCH 3
Running average loss: 0.415883
Elapsed time 106.2205502986908 seconds

EPOCH 4
Running average loss: 0.384447
Elapsed time 107.26095294952393 seconds

EPOCH 5
Running average loss: 0.384038
Elapsed time 107.32022714614868 seconds

EPOCH 6
Running average loss: 0.393268
Elapsed time 107.60648727416992 seconds

EPOCH 7
Running average loss: 0.403062
Elapsed time 107.50295424461365 seconds

EPOCH 8
Running average loss: 0.386139
Elapsed time 107.26458477973938 seconds

EPOCH 9
Running average loss: 0.376185
Elapsed time 107.23961544036865 seconds

EPOCH 10
Running average loss: 0.395370
Elapsed time 107.49779915809631 seconds

EPOCH 11
Running average loss: 0.373344
Elapsed time 107.35019207000732 seconds

EPOCH 12
Running average loss: 0.372232
Elapsed time 107.40108442306519 seconds

EPOCH 13
Running average loss: 0.38242

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
with clock():
    model.fit(trainloader, None, optimizer=optimizer, num_epochs=8)
save_best_path = base_dir+'rnn_v2_predictor_lr2_weights.mdl'
torch.save(model.state_dict(), save_best_path)


EPOCH 1
Running average loss: 0.371828
Elapsed time 107.35065841674805 seconds

EPOCH 2
Running average loss: 0.366410
Elapsed time 107.13260817527771 seconds

EPOCH 3
Running average loss: 0.360923
Elapsed time 107.18335103988647 seconds

EPOCH 4
Running average loss: 0.353547
Elapsed time 107.15616416931152 seconds

EPOCH 5
Running average loss: 0.364152
Elapsed time 107.2029218673706 seconds

EPOCH 6
Running average loss: 0.363673
Elapsed time 107.23570728302002 seconds

EPOCH 7
Running average loss: 0.364665
Elapsed time 107.27262473106384 seconds

EPOCH 8
Running average loss: 0.354986
Elapsed time 107.22689461708069 seconds
Elapsed time 857.7650198936462 seconds


In [11]:
#save_best_path = base_dir+'rnn_v2_predictor_lr2_weights.mdl'
#loss_func = nn.L1Loss()
#model = rnn.RNN(loss_func=loss_func).cuda()
#model.load_state_dict(torch.load(save_best_path))

In [52]:
outputs, targets, sequences = model.predict(testloader)

In [53]:
predictions = scaler.inverse_transform(outputs.data.cpu().numpy().squeeze().T).T

In [51]:
predictions

array([[  2.08768597e+01,   2.06833591e+01,   2.01083450e+01, ...,
          2.27052174e+01,   2.24966240e+01,   2.23551865e+01],
       [  2.13238945e+01,   2.04893436e+01,   2.20704422e+01, ...,
          2.20366802e+01,   2.29790096e+01,   2.40018997e+01],
       [  7.81798315e+00,   5.79528189e+00,   5.54005861e+00, ...,
          4.86465740e+00,   4.68661642e+00,   4.54093695e+00],
       ..., 
       [ -1.26851350e-03,  -1.82658434e-03,  -2.10051611e-03, ...,
         -9.74615663e-03,  -9.88163427e-03,  -9.98216122e-03],
       [ -1.26851350e-03,  -1.82658434e-03,  -2.10051611e-03, ...,
         -9.74615663e-03,  -9.88163427e-03,  -9.98216122e-03],
       [ -1.26851350e-03,  -1.82658434e-03,  -2.10051611e-03, ...,
         -9.74615663e-03,  -9.88163427e-03,  -9.98216122e-03]], dtype=float32)

In [49]:
fp = '../data/submissions/rnn_v2.csv'
submissions.write_submission(predictions, fp)